In [112]:
import nidaqmx 
import atiiaftt as ati
import time
import numpy as np
from nidaqmx.stream_readers import AnalogMultiChannelReader
import matplotlib.pyplot as plt
import pandas as pd
from nidaqmx.constants import AcquisitionType

In [113]:
CALFILEPATH = "..\FT44764\FT44764.cal"
INDEX = 1

with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai5")
    
    readings = task.read()

sampleTT = [0,0,0,0,0,0]

sensor = ati.FTSensor()
cal = sensor.createCalibration(CALFILEPATH, INDEX)
sensor.setForceUnits("N".encode("utf-8"))
sensor.setTorqueUnits("N-m".encode("utf-8"))
sensor.setToolTransform(sampleTT, "mm".encode("utf-8"), "deg".encode("utf-8"))

DURATION = 5

In [114]:
current_time = time.time()
readings_inside_loop = []
while time.time() < current_time+DURATION:
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai5")

        readings_inside_loop.append(sensor.convertToFt(task.read()))
readings_inside_loop = np.array(readings_inside_loop)

print("Shape size: ", readings_inside_loop.shape)
std_readings_inside_loop = readings_inside_loop.transpose()[0].std()
print("Standard Deviation of Force X (N): ", std_readings_inside_loop)
avg_readings_inside_loop = readings_inside_loop.transpose()[0].mean()
print("Average of Force X (N): ", avg_readings_inside_loop)

Shape size:  (182, 6)
Standard Deviation of Force X (N):  0.003857721838954545
Average of Force X (N):  1.1917889144394425


In [115]:
current_time = time.time()
readings_outside_loop = []
with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    task.ai_channels.add_ai_voltage_chan("Dev1/ai5")
    while time.time() < current_time+DURATION:
        readings_outside_loop.append(sensor.convertToFt(task.read()))
readings_outside_loop = np.array(readings_outside_loop)


print("Shape size: ", readings_outside_loop.shape)
std_readings_outside_loop = readings_outside_loop.transpose()[0].std()
print("Standard Deviation of Force X (N): ", std_readings_outside_loop)
avg_readings_outside_loop = readings_outside_loop.transpose()[0].mean()
print("Average of Force X (N): ", avg_readings_outside_loop)

Shape size:  (216, 6)
Standard Deviation of Force X (N):  0.003573190425981459
Average of Force X (N):  1.191310512246909


In [116]:
readings_multi_read_one = []
with nidaqmx.Task() as read_task:
    read_task = nidaqmx.Task()
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai5")
    reader = AnalogMultiChannelReader(read_task.in_stream,)
    values_read = np.zeros(6, dtype=np.float64)



    current_time = time.time()
    while time.time() < current_time+DURATION:
        reader.read_one_sample(values_read)
        
        readings_multi_read_one.append(sensor.convertToFt(values_read.tolist()))

readings_multi_read_one = np.array(readings_multi_read_one)


print("Shape size: ", readings_multi_read_one.shape)
std_readings_multi_read_one = readings_multi_read_one.transpose()[0].std()
print("Standard Deviation of Force X (N): ", std_readings_multi_read_one)
avg_readings_multi_read_one = readings_multi_read_one.transpose()[0].mean()
print("Average of Force X (N): ", avg_readings_multi_read_one)

Shape size:  (215, 6)
Standard Deviation of Force X (N):  0.0039006248038501914
Average of Force X (N):  1.1904202987981396


In [117]:
readings_multi_read_many = []
with nidaqmx.Task() as read_task:
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai2")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai3")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai4")
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai5")

    sample_rate = 6400
    samples_to_acq = DURATION * sample_rate
    values_read = np.zeros((6, samples_to_acq), dtype=np.float64)
    cont_mode = AcquisitionType.CONTINUOUS
    read_task.timing.cfg_samp_clk_timing(sample_rate, sample_mode = cont_mode, samps_per_chan = samples_to_acq)               
    reader = AnalogMultiChannelReader(read_task.in_stream)

    current_time = time.time()
    reader.read_many_sample(data=values_read, number_of_samples_per_channel=samples_to_acq, timeout=11)
    test_time = time.time()
    print("time passed: ", test_time-current_time,"(s)")

readings_multi_read_many = np.array([np.array(channel) for channel in values_read]).transpose()
readings_multi_read_many = np.array([sensor.convertToFt(reading.tolist()) for reading in readings_multi_read_many])

print("Shape size: ", readings_multi_read_many.shape)
std_readings_multi_read_many = readings_multi_read_many.transpose()[0].std()
print("Standard Deviation of Force X (N): ", std_readings_multi_read_many)
avg_readings_multi_read_many = readings_multi_read_many.transpose()[0].mean()
print("Average of Force X (N): ", avg_readings_multi_read_many)

time passed:  5.037401914596558 (s)
Shape size:  (32000, 6)
Standard Deviation of Force X (N):  0.003944422318397909
Average of Force X (N):  1.188775642607361


c:\users\andyb\appdata\local\programs\python\python310\lib\site-packages\nidaqmx\task.py:97: ResourceWarning: Task of name "_unnamedTask<73D>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(


In [118]:
FOLDER_DIR = "..\Joey\Record_Data"

demo_df = pd.read_csv(FOLDER_DIR+"\Demo\Sensor_2.csv")

demo_df.drop(demo_df.columns[6:], axis=1, inplace=True)

print("Shape size: ", demo_df.shape)
print("Standard Deviation of Force X (N): ", demo_df["Force X (N)"].std())
print("Average of Force X (N): ", demo_df["Force X (N)"].mean())

Shape size:  (500, 6)
Standard Deviation of Force X (N):  0.0018800538336572557
Average of Force X (N):  1.1916512076854706
